In [1]:
import pickle
import numpy as np
import elasticSearchTest as est

In [2]:
from sklearn.model_selection import train_test_split
def splitSet(vec,y):
    return train_test_split(  vec, y, test_size=0.2, random_state=2)

In [3]:
yFR = []
yEN = []
res = est.getDocuments("rss",{"match_all": {} })
hits = res["hits"]["hits"]
categories = ["BUSINESS","SPORT","HEALTH","ART","SCIENCE"]

for hit in hits:
    if hit["_source"]["data"] != None:
        if hit["_source"]["language"] == "fr":      
            yFR.append(categories.index(hit["_source"]["Catégorie_du_flux"]))
        else:
            yEN.append(categories.index(hit["_source"]["Catégorie_du_flux"]))

In [4]:
print("loading vectorizer FR...")
vecFR = pickle.load(open("vec/vecFR.pickle", "rb"))
print("vectorizer FR loaded!")
X_train_FR, X_test_FR, y_train_FR, y_test_FR = splitSet(vecFR,yFR)
print("dataset FR created!")

loading vectorizer FR...
vectorizer FR loaded!
dataset FR created!


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim


if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(860498, 1024)
        self.linear2 = nn.Linear(1024, 512)
        self.linear3 = nn.Linear(512, 5)
        self.logsoft = nn.LogSoftmax(dim=1)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.drop(x)
        
        x = self.relu(self.linear2(x))
        x = self.drop(x)
        
        x = self.logsoft(self.linear3(x))
        
        return x

networkFR = Model() # create network object
networkEN = Model() # create network object
networkFR.to(device)
# networkEN.to(device)


criterion = nn.CrossEntropyLoss() # define loss. Using negative log likelyhood loss here.
optimizer = torch.optim.Adam(networkFR.parameters(), lr=0.0001) # define optimization algorithm. Here we'll use Adam


n_epoch = 75 # number of epochs



In [6]:

train_data = []
for i in range(len(X_train_FR)):
    train_data.append([X_train_FR[i], yFR[i]])
trainloader=torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, num_workers=2)


In [8]:
train_losses = [] # set of training loss for each epoch
valid_losses = [] # set of validation loss for each epoch

valid_loss_min = np.inf

networkFR.train() # training loop

for i in range(n_epoch):
    train_loss = 0
    valid_loss = 0
    print("{} / {}".format(i,n_epoch), end="\r") 
    print()
    
    for idx,batch in enumerate(trainloader):
        print("{} / {}".format(idx,len(trainloader)), end="\r") 
        data, labels = batch
        optimizer.zero_grad()

        logits = networkFR(data.float())

        loss = criterion(logits, labels)

        train_loss += loss

        loss.backward()

        optimizer.step()
    
    #print(loss.device)

    train_losses.append(train_loss)

0 / 75


KeyboardInterrupt: 